In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset

import torch
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [2]:
# class MNIST_Dataset(Dataset):
#   def __init__(self, csv_file):
#     self.data = pd.read_csv(csv_file)

#   def __getitem__(self, index):
#     r = self.data.iloc[index]
#     label, image = r
#     return label, image
  
#   def __len__(self):
#     return len(self.data)

In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_set = datasets.MNIST(
    root = '.data/'
    ,train=True
    ,download = True
    ,transform = transforms.Compose([
                  transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(train_set,
    batch_size=batch_size, shuffle=True, **kwargs)

test_set = datasets.MNIST(
    root = '.data/'
    ,train=False
    ,download = True
    ,transform = transforms.Compose([
                  transforms.ToTensor()
    ])
)

test_loader = torch.utils.data.DataLoader(test_set,
    batch_size=batch_size, shuffle=True, **kwargs)



In [ ]:
max_rand_no = 10
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #  28*28  32  3*3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #  28*28  64  5*5
        self.pool1 = nn.MaxPool2d(2, 2)  # 14*14  64  10*10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)  #  14*14  128  12*12 
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #  14*14  256  14*14
        self.pool2 = nn.MaxPool2d(2, 2)  # 7*7  256  28*28
        self.conv5 = nn.Conv2d(256, 512, 3)  # 5*5  512  30*30
        self.conv6 = nn.Conv2d(512, 1024, 3)  # 3*3  1024  32*32
        self.conv7 = nn.Conv2d(1024, 10, 3)   # 1*1  10  34*34  # how come receptive field be more than the image size?

    def forward(self, x, y): 
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x)))) 
        x = (self.conv7(x))  
        x = x.view(-1, 10)
        # x = F.log_softmax(x)
        x_for_sum = torch.argmax(x, dim=1)  # label, _ = x 
        sum_x_y = x_for_sum + y
        sum_x_y_encoded = F.one_hot(sum_x_y, num_classes=max_rand_no*2)
        return x, sum_x_y_encoded

model = Net().to(device)
for name, param in model.named_parameters():
  print(name, '\t\t', param.shape)

In [ ]:
import random
import sys
max_rand_no = 10


from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        randomlist = []
        for i in range(0,data.size()[0]):
          rand_no = random.randint(0,max_rand_no)
          randomlist.append(rand_no)
        randomtensor = torch.as_tensor(randomlist)
        randomtensor = randomtensor.cuda()

        output, sum_rand_image = model(data, randomtensor)  # sum_randomtensor
        loss = F.cross_entropy(output, target)
        sum_rand_image = sum_rand_image.type(torch.FloatTensor)
        target_2 = (target+randomtensor)
        sum_rand_image = sum_rand_image.cuda()

        loss_2 = F.cross_entropy(sum_rand_image, target_2)
        loss = loss+loss_2
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    loss_2 = 0
    correct = 0
    correct_2 = 0 
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            randomlist = []
            for i in range(0,data.size()[0]):
              rand_no = random.randint(0,max_rand_no)
              randomlist.append(rand_no)
            randomtensor = torch.as_tensor(randomlist)
            randomtensor = randomtensor.cuda()
            
            output, sum_rand_image = model(data, randomtensor)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            sum_rand_image = sum_rand_image.type(torch.FloatTensor)
            target_2 = (target+randomtensor)
            sum_rand_image = sum_rand_image.cuda()
            loss_2 = F.cross_entropy(sum_rand_image, target_2, reduction='sum').item()
            loss = test_loss+loss_2

            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            pred_2 = sum_rand_image.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct_2 += pred_2.eq(target_2.view_as(pred_2)).sum().item()
    test_loss /= len(test_loader.dataset)
    loss_2 /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        loss_2, correct_2, len(test_loader.dataset),
        100. * correct_2 / len(test_loader.dataset)))
    print('\nTest set: Average loss: {:.4f}'.format(loss))

In [ ]:
for epoch in range(1, 5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)